# Statistically Testing 2P Responses to Stimulus
In most analyses, some form of *inclusion criteria* is used to select neurons that are "responsive" to the stimulus conditions presented. There are no universally agreed upon inclusion criteria for this type of selection. In [{cite}`Mesa2021`](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8114876/), it is demonstrated that the choice of inclusion criteria can dramatically affect what neurons are selected as responsive. This notebook does a similar demonstration, using the same five inclusion criteria on Ophys dF/F recordings to select responsive neurons from one experimental session. It can be seen that very different selections are made depending on the criteria used. This also underscores how different critera might be more or less appropriate for the type of stimulus and the type of measurements being used for analysis. We are sharing these so that this comparison can be reproduced on data and so results can more effectively use this choice to make their results more robust. For more information about dF/F data, see [Visualizing 2P Responses to Stimulus](../visualization/visualize_2p_responses.ipynb).

### Environment Setup

In [ ]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !git clone https://github.com/AllenInstitute/openscope_databook.git
    %cd openscope_databook
    %pip install -e .

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from dandi_utils import dandi_download_open
from matplotlib_venn import venn2
from scipy import interpolate
from scipy.stats import ttest_ind

%matplotlib inline

### Downloading Ophys File
Change the values below to download the file you’re interested in. In this example, we use the dF/F trace from an Ophys NWB File, so you'll have to choose one with the same kind of data. Set `dandiset_id` and `dandi_filepath` to correspond to the dandiset id and filepath of the file you want. If you’re accessing an embargoed dataset, set `dandi_api_key` to your DANDI API key.

In [ ]:
dandiset_id = "000336"
dandi_filepath = "sub_1170622509/sub_1170622509sess_1194683535/sub_1170622509+sess_1194683535+exp_1194752582_ophys.nwb"
download_loc = "."
dandi_api_key = os.environ["DANDI_API_KEY"]

In [ ]:
nwb = dandi_download_open(dandiset_id, dandi_filepath, download_loc, dandi_api_key=dandi_api_key)

### Getting 2P Data and Stimulus Data
Below, the fluorescence traces and timestamps are read from the file's `Processing` section. In this notebook, we will be interested in the DFF trace specifically. Note that the exact format to access these traces can vary between newer and older NWB files, so some adjustments may be necessary. Additionally, the stimulus data is also read from the NWB file's `Intervals` section. Stimulus information is stored as a series of tables depending on the type of stimulus shown in the session. One such table is displayed below, as well as the names of all the tables.

In [ ]:
dff = nwb.processing["ophys"]["dff"]
dff_trace = dff.roi_response_series["traces"].data
dff_timestamps = dff.roi_response_series["traces"].timestamps

# accessing the above data may look different for older nwb files, like the following
# dff_trace = dff.roi_response_series["RoiResponseSeries"].data
# dff_timestamps = dff.roi_response_series["RoiResponseSeries"].timestamps

print(dff_trace.shape)
print(dff_timestamps.shape)

### Fluorescence Interpolation
Because we cannot be certain that the DFF trace is at a perfectly regular Hz, we will interpolate it to be able to compare stimulus timestamps to their approximate DFF times. After you have a valid list of stimulus timestamps, you can generate a linearly-spaced timestamp array called `time_axis`, and interpolate the dF/F data along it, making interpolated DFF data called `interp_dff`. This should be a 2D array with dimensions `time` and `roi`, where rois are the different cells whose fluorescence is measured. Here, the timestamps are interpolated to 10 Hertz, but you can change this by setting `interp_hz`.

In [ ]:
interp_hz = 10

In [ ]:
print(dff_trace.shape)
print(dff_timestamps.shape)

In [ ]:
# generate regularly-space x values and interpolate along it
time_axis = np.arange(dff_timestamps[0], dff_timestamps[-1], step=(1/interp_hz))
interp_dff = []

# interpolate channel by channel to save RAM
for channel in range(dff_trace.shape[1]):
    f = interpolate.interp1d(dff_timestamps, dff_trace[:,channel], axis=0, kind="nearest", fill_value="extrapolate")
    interp_dff.append(f(time_axis))

interp_dff = np.array(interp_dff)

print(interp_dff.shape)

### Selecting Stimulus Times
Different types of stimulus require different kinds of inclusion criteria. Since the available stimulus tables vary significantly depending which NWB file and which experimental session you're analyzing, you'll have to adjust some values below. First, select which stimulus table you want by changing the key used below in `nwb.intervals`. The list of stimulus table names is printed below to inform this choice. Additionally, you'll have to modify the function `stim_select` to select the stimulus times you want to use. In this example, a natural movie of worms is the selected stimulus, and frame 40 is chosen somewhat arbitrarily.

In [ ]:
stimulus_names = list(nwb.intervals.keys())
print(stimulus_names)

In [ ]:
stim_table = nwb.intervals["movie_worms_fwd_presentations"]
print(stim_table.colnames)
stim_table[:10]

In [ ]:
### select start times from table that fit certain critera here

# stim_select = lambda row: True
stim_select = lambda row: float(row.frame) == 40
all_stim_times = [float(stim_table[i].start_time) for i in range(len(stim_table)) if stim_select(stim_table[i])]
len(all_stim_times)

### Getting Stimulus Time Windows
Now that you have your interpolated dF/F data and the selected stimulus timestamps, you identify the windows of time in the dF/F data that exist around each stimulus event. Since the dF/F data have been interpolated, we can easily translate timestamps to indices within the LFP trace array. Set `window_start_time` to be a negative number, representing the seconds before the stimulus event and `window_end_time` to be number of seconds afterward. Then the `windows` array will be generated as a set of slices of the `interp_dff` trace by using `interp_hz` to convert seconds to array indices. These windows can be used for the application of the various inclusion criteria.

In [ ]:
window_start_time = -2
window_end_time = 3

In [ ]:
# validate window bounds
if window_start_time > 0:
    raise ValueError("start time must be non-positive number")
if window_end_time <= 0:
    raise ValueError("end time must be positive number")
    
# get event windows
windows = []
window_length = int((window_end_time-window_start_time) * interp_hz)

for stim_ts in all_stim_times:
    # convert time to index
    start_idx = int( (stim_ts + window_start_time - dff_timestamps[0]) * interp_hz )
    end_idx = start_idx + window_length
 
    # bounds checking
    if start_idx < 0 or end_idx > interp_dff.shape[1]:
        continue
        
    windows.append(interp_dff[:,start_idx:end_idx])
    
if len(windows) == 0:
    raise ValueError("There are no windows for these timestamps")

windows = np.array(windows) * 100 # x100 to convert values to dF/F percentage
print(windows.shape)

### Defining Plotting Methods
Here are a few methods to display the stimulus response windows. These will be used to display the ROIs selected by each inclusion critera.

In [ ]:
def show_dff_response(ax, dff, window_start_time, window_end_time, aspect="auto", vmin=None, vmax=None, yticklabels=[], skipticks=1, xlabel="Time (s)", ylabel="ROI", cbar=True, cbar_label=None):
    if len(dff) == 0:
        print("Input data has length 0; Nothing to display")
        return

    img = ax.imshow(dff, aspect=aspect, extent=[window_start_time, window_end_time, 0, len(dff)], vmin=vmin, vmax=vmax)
    if cbar:
        fig.colorbar(img, shrink=0.5, label=cbar_label)

    ax.plot([0,0],[0, len(dff)], ":", color="white", linewidth=1.0)

    if len(yticklabels) != 0:
        ax.set_yticks(range(len(yticklabels)))
        ax.set_yticklabels(yticklabels, fontsize=8)

        n_ticks = len(yticklabels[::skipticks])
        ax.yaxis.set_major_locator(plt.MaxNLocator(n_ticks))

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

In [ ]:
def show_many_responses(windows, rows, cols, title=None, subplot_title="", xlabel=None, ylabel=None, cbar_label=None, vmin=0, vmax=100):
    # handle case with no input data
    if len(windows) == 0:
        print("Input data has length 0; Nothing to display")
        return
    # handle cases when there aren't enough windows for number of rows
    if len(windows) < rows*cols:
        rows = (len(windows) // cols) + 1

    fig, axes = plt.subplots(rows, cols, figsize=(2*cols+2, 2*rows+2), layout="constrained")
    # handle case when there's only one row
    if len(axes.shape) == 1:
        axes = axes.reshape((1, axes.shape[0]))
    
    for idx in range(rows*cols):
        ax_row = int(idx // cols)
        ax_col = idx % cols
        ax = axes[ax_row][ax_col]
        
        if idx > len(windows)-1:
            ax.set_visible(False)
            continue

        window = windows[idx]
        show_dff_response(ax, window, window_start_time, window_end_time, xlabel=xlabel, ylabel=ylabel, cbar=False, vmin=vmin, vmax=vmax)
        ax.set_title(f"{subplot_title} {idx}")
        if ax_row != rows-1:
            ax.get_xaxis().set_visible(False)
        if ax_col != 0:
            ax.get_yaxis().set_visible(False)


    fig.suptitle(title)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    colorbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm), ax=axes, shrink=1.5/rows, label=cbar_label)


### Aligning Time Windows
The stimulus windows are averaged across all stimulus trials to get an average view of how the ROIs respond. Use this to compare to the mean responses of the selected neurons after each inclusion criteria.

In [ ]:
mean_window = np.mean(windows, axis=0)
print(mean_window.shape)
fig, ax = plt.subplots()
show_dff_response(ax, mean_window, window_start_time, window_end_time, vmin=0, vmax=100, cbar_label="$\Delta$F/F (%)")
ax.set_title("Mean Response Window")
ax.set_xlabel("time (s)")
ax.set_ylabel("ROI (neuron)")

### Showing Response Windows

In [ ]:
show_many_responses(windows,
                    3,
                    8,
                    title="Responses to each trial",
                    subplot_title="Trial",
                    xlabel="time (s)",
                    ylabel="ROI (neuron)",
                    cbar_label="$\Delta$F/F (%)")

In [ ]:
neuronwise_windows = np.swapaxes(windows,0,1)
show_many_responses(neuronwise_windows,
                    3,
                    8,
                    title="Response of ROIs across trials",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")

### Getting Stimulus Window Info
We calculate a few values which will be applicable to multiple inclusion criteria.

In [ ]:
# get the index within the window that stimulus occurs (time 0)
stimulus_onset_idx = int(-window_start_time * interp_hz)
baseline = windows[:,:,0:stimulus_onset_idx]
evoked_responses = windows[:,:,stimulus_onset_idx:]

print(stimulus_onset_idx)
print(baseline.shape)
print(evoked_responses.shape)

### Inclusion Criteria

#### Inclusion Criterion 1
The maximum value of the mean evoked response is >10% [{cite}`Sun2016`](https://rdcu.be/dbFb9)

In [ ]:
mean_evoked_responses = np.mean(evoked_responses, axis=0)
max_mean_evoked_responses = np.max(mean_evoked_responses, axis=1)
IC1_selected_rois = np.where(max_mean_evoked_responses > 10)[0]
print(f"Selected ROIs {IC1_selected_rois}")

In [ ]:
plt.hist(max_mean_evoked_responses, bins=100)
plt.ylabel("# ROIs")
plt.axvline(x=10, color="red", linestyle=":")
plt.xlabel("Maximum $\Delta$F/F (%) of mean evoked response")
plt.show()

In [ ]:
print(f"{len(IC1_selected_rois)} / {neuronwise_windows.shape[0]} ROIs selected")
show_many_responses(neuronwise_windows[IC1_selected_rois],
                    3,
                    8,
                    title="Neuronwise Responses of Included ROIs (Criterion 1)",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")

#### Inclusion Criterion 2
In 50% of trials, the evoked response is A) larger than 3x the SD of the baseline, and B) larger than 5% dF/F [{cite}`Roth2012`](https://www.jneurosci.org/content/jneuro/32/28/9716.full.pdf)

In [ ]:
all_baseline_sds = np.std(baseline, axis=2)
print(all_baseline_sds.shape)

max_responses = np.max(evoked_responses, axis=2)
print(max_responses.shape)

# rois in each trial that respond more than 3x the SD of baseline
deviant_responses = max_responses > 3*all_baseline_sds
# rois in each trial that response more than 5% dff
large_responses = max_responses > 5
# rois in each trial that do both of the above
sig_responses = deviant_responses & large_responses

# rois that have significant responses in 50% or more trials
half_trials = sig_responses.shape[0] / 2
IC2_selected_rois = np.where( np.sum(sig_responses, axis=0) > half_trials )[0]
print(f"Selected ROIs {IC2_selected_rois}")

In [ ]:
z_scores = np.mean(max_responses / all_baseline_sds, axis=0)
plt.hist(z_scores, bins=50)
plt.xlabel("Z-score")
plt.ylabel("# ROIs")
# plt.axvline(x=3, color="red", linestyle=":")
plt.show()

In [ ]:
print(f"{len(IC1_selected_rois)} / {neuronwise_windows.shape[0]} ROIs selected")
show_many_responses(neuronwise_windows[IC2_selected_rois],
                    3,
                    8,
                    title="Neuronwise Responses of Included ROIs (Criterion 2)",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")

#### Inclusion Criterion 3
Paired T-test (p < 0.05) with Bonferroni correction, comparing the mean baseline to the mean evoked response [{cite}`Andermann2011`](https://pubmed.ncbi.nlm.nih.gov/22196337/)

In [ ]:
mean_trial_responses = np.mean(evoked_responses, axis=2)
mean_trial_baselines = np.mean(baseline, axis=2)

n = mean_trial_responses.shape[0]
t,p = ttest_ind(mean_trial_responses, mean_trial_baselines)
IC3_selected_rois = np.where(p < 0.05 / n)[0]
print(f"Selected ROIs {IC3_selected_rois}")

In [ ]:
np.sort(p)

In [ ]:
roi = 15
baseline_dist = mean_trial_baselines[:,roi]
response_dist = mean_trial_responses[:,roi]
max_val = max(np.maximum(baseline_dist, response_dist)) + 1
min_val = min(np.minimum(baseline_dist, response_dist))

plt.hist(baseline_dist, bins=50, range=(min_val, max_val), alpha=0.5, color="blue", label="baseline")
plt.hist(response_dist, bins=50, range=(min_val, max_val), alpha=0.5, color="orange", label="evoked")
plt.xlabel("Mean response $\Delta$F/F (%)")
plt.ylabel("# Trial responses")
plt.title(f"Baseline and Evoked Responses for ROI {roi}")
plt.show

In [ ]:
hist, _, _  = plt.hist(p, bins=50)
plt.xlabel("P-value")
plt.ylabel("# ROIs")
thresh = 0.05/n
plt.axvline(x=thresh, color="red", linestyle=":")
plt.show()

In [ ]:
print(f"{len(IC1_selected_rois)} / {neuronwise_windows.shape[0]} ROIs selected")
show_many_responses(neuronwise_windows[IC3_selected_rois],
                    3,
                    8,
                    title="Neuronwise Responses of Included ROIs (Criterion 3)",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")

#### Inclusion Criterion 4
A) The mean response to any stimulus condition is is > 6% dF/F and B) reliability > 1 [{cite}`Marshel2012`](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3248795/). It can be seen that in this example, no ROIs are selected. This is probably due to no ROIs having reliable responses to the stimulus.

In [ ]:
mean_trial_responses = np.mean(evoked_responses, axis=2)
response_means = np.mean(mean_trial_responses, axis=0)
response_sds = np.std(mean_trial_responses, axis=0)

mean_trial_baselines = np.mean(baseline, axis=2)
baseline_means = np.mean(mean_trial_baselines, axis=0)
baseline_sds = np.std(mean_trial_baselines, axis=0)

reliabilities = (response_means - baseline_means) / (response_sds + baseline_sds)

sig_reliabilities = reliabilities > 1
large_responses = response_means > 6
IC4_selected_rois = np.where(sig_reliabilities & large_responses)[0]
print(f"Selected ROIs {IC4_selected_rois}")

In [ ]:
plt.hist(reliabilities, bins=50)
plt.ylabel("# ROIs")
plt.xlabel("Reliability")
plt.axvline(x=1, color="red", linestyle=":")
plt.show()

In [ ]:
print(f"{len(IC1_selected_rois)} / {neuronwise_windows.shape[0]} ROIs selected")
show_many_responses(neuronwise_windows[IC4_selected_rois],
                    3,
                    8,
                    title="Neuronwise Responses of Included ROIs (Criterion 4)",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")

#### Inclusion Criterion 5
The maximum evoked response to any stimulus condition is > 4% dF/F [{cite}`Tohmi2014`](https://pubmed.ncbi.nlm.nih.gov/24583013/). This criterion is clearly the most permissive, as it includes all ROIs for this dataset.

In [ ]:
max_responses = np.max(evoked_responses, axis=(0,2))
IC5_selected_rois = np.where(max_responses > 4)[0]
print(f"Selected ROIs {IC5_selected_rois}")

In [ ]:
plt.hist(max_responses, bins=100)
plt.ylabel("# ROIs")
plt.xlabel("max $\Delta$F/F (%) of Evoked responses")
plt.axvline(x=4, color="red", linestyle=":")
plt.show()

In [ ]:
print(f"{len(IC1_selected_rois)} / {neuronwise_windows.shape[0]} ROIs selected")
show_many_responses(neuronwise_windows[IC5_selected_rois],
                    3,
                    8,
                    title="Neuronwise Responses of Included ROIs (Criterion 5)",
                    subplot_title="ROI",
                    xlabel="time (s)",
                    ylabel="trial",
                    cbar_label="$\Delta$F/F (%)")